In [4]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv('../.env')
engine = create_engine(os.getenv('DB_URL'))

dates = pd.date_range('2010-01-01', '2025-12-31', freq='D')

dim_date = pd.DataFrame({
    'date_key':      dates.strftime('%Y%m%d').astype(int),
    'full_date':     dates.date,
    'day_of_week':   dates.day_name(),
    'day_num':       dates.dayofweek + 1,
    'week_num':      dates.isocalendar().week.astype(int),
    'month_num':     dates.month,
    'month_name':    dates.month_name(),
    'quarter_num':   dates.quarter,
    'fiscal_year':   dates.year,
    'fiscal_quarter': 'Q' + dates.quarter.astype(str) + '-' + dates.year.astype(str),
    'is_weekend':    dates.dayofweek >= 5
})

dim_date.to_sql('dim_date', engine, schema='gold',
                if_exists='replace', index=False)

with engine.connect() as conn:
    n = conn.execute(text('SELECT COUNT(*) FROM gold.dim_date')).scalar()
    print(f'✓ gold.dim_date: {n:,} rows')

# Quick sanity check
print(dim_date[dim_date['full_date'] == pd.Timestamp('2011-11-25').date()])
# Should show date_key = 20111125, is_weekend = False

✓ gold.dim_date: 5,844 rows
            date_key   full_date day_of_week  day_num  week_num  month_num  \
2011-11-25  20111125  2011-11-25      Friday        5        47         11   

           month_name  quarter_num  fiscal_year fiscal_quarter  is_weekend  
2011-11-25   November            4         2011        Q4-2011       False  
